In [1]:
import os
os.chdir(r"C:\Users\Laura\Desktop\saeb_microdados_dashboard")

In [2]:

import pandas as pd
from src.ingestion.read_alunos import read_alunos
from src.dictionaries.alunos_maps import (
    map_categorias_basicas_alunos,
    map_geografia_alunos,
    map_questionario_socioeconomico,
    map_opiniao_matematica,
    
)
from src.cleaning.alunos_cleaning import (
    clean_alunos_data,
    drop_missing_essential,
    clean_zero_to_nan,
)

In [6]:
df_alunos = read_alunos(1995)
pd.set_option("display.max_rows", None)

df_alunos = map_categorias_basicas_alunos(df_alunos)
df_alunos = map_geografia_alunos(df_alunos)
df_alunos = map_questionario_socioeconomico(df_alunos)
df_alunos = map_opiniao_matematica(df_alunos)
df_alunos = clean_alunos_data(df_alunos)
df_alunos = drop_missing_essential(df_alunos)
df_alunos = clean_zero_to_nan(df_alunos)

In [8]:
df_alunos.head()

,ano,serie,id_aluno,idade,estrato,upa,dep_adm,local,rede,uf,regiao,peso_ac,proficiencia,data_nascimento,sexo,raca,instrucao_pai,instrucao_mae,mora_com,trabalha,compra_jornal,horas_tv,conversa_escola,estudo_interrompido,faltas,anos_1grau,anos_2grau,supletivo,opiniao_teste,tempo_licao,gosta_mat,entende_professor,genero_mat,utilidade_mat,esforco_professor
0,1995,3º Ano,15,20.0,110,1,Municipal,Urbana,Pública,Rondônia,Norte,26.0,204.7948,090475,Feminino,Pardo(a),Nunca frequentou,Nunca frequentou,1.0,Sim (8h/dia),1.0,2.0,2.0,3.0,5 dias,1.0,2.0,2.0,Muito difícil,1.0,Concordo em parte,Concordo em parte,Concordo totalmente,Concordo totalmente,Não sei
1,1995,3º Ano,10,24.0,110,2,Municipal,Urbana,Pública,Rondônia,Norte,37.0,218.7780,180271,Masculino,Preto(a),Primário (1ª-4ª),Primário (1ª-4ª),2.0,Sim (8h/dia),5.0,3.0,2.0,5.0,> 30 dias,2.0,1.0,2.0,Difícil,3.0,Concordo em parte,Discordo em parte,Concordo em parte,Concordo totalmente,Concordo em parte
2,1995,3º Ano,27,18.0,110,6,Federal,Urbana,Particular,Rondônia,Norte,14.0,329.1833,280876,Feminino,Preto(a),Superior,Ginásio (5ª-8ª),1.0,Não,4.0,2.0,2.0,1.0,15 dias,1.0,3.0,2.0,Mais ou menos,5.0,Concordo totalmente,Concordo em parte,NaN,Concordo em parte,Concordo em parte
3,1995,3º Ano,9,NaN,120,3,Municipal,Urbana,Pública,Acre,Norte,10.0,210.8740,NaN,Feminino,Pardo(a),Primário (1ª-4ª),Primário (1ª-4ª),5.0,Sim (4-6h/dia),5.0,4.0,2.0,1.0,5 dias,1.0,3.0,2.0,Muito difícil,1.0,Concordo totalmente,Não sei,Concordo totalmente,Concordo totalmente,NaN
4,1995,3º Ano,31,17.0,120,4,Federal,Urbana,Particular,Acre,Norte,9.0,309.3175,031277,Feminino,Pardo(a),Primário (1ª-4ª),Primário (1ª-4ª),2.0,Não,5.0,3.0,2.0,1.0,5 dias,1.0,3.0,2.0,Mais ou menos,4.0,Concordo em parte,Discordo em parte,Não sei,Discordo em parte,Não sei


In [7]:
df_alunos.info()

<class 'pandas.DataFrame'>
RangeIndex: 9049 entries, 0 to 9048
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  9049 non-null   int64  
 1   serie                9049 non-null   str    
 2   id_aluno             9049 non-null   int64  
 3   idade                8489 non-null   float64
 4   estrato              9049 non-null   int64  
 5   upa                  9049 non-null   int64  
 6   dep_adm              9049 non-null   str    
 7   local                9049 non-null   str    
 8   rede                 8924 non-null   str    
 9   uf                   9049 non-null   str    
 10  regiao               9049 non-null   str    
 11  peso_ac              9049 non-null   float64
 12  proficiencia         9049 non-null   float64
 13  data_nascimento      8489 non-null   str    
 14  sexo                 8931 non-null   str    
 15  raca                 8919 non-null   str    
 16 